In [60]:
import sqlite3

import pandas as pd
import numpy as np
from urllib.request import urlopen  
import json

getkey = ! cat ../../apikey.txt
apikey = getkey[0]

from fhelp import get_json, fmp_todf

In [61]:
#get list of tickers (just dow until everything is set up)
dow_url = "https://financialmodelingprep.com/api/v3/dowjones_constituent?apikey=600e501fe324b2f1cbb2aec5b390b6db"
dow_ticks = get_json(dow_url)
dow_tickers = [tick['symbol'] for tick in dow_ticks]
tickers = dow_tickers

In [3]:
tickers += ['SECO']

In [4]:
#connect to dv(setup if new)
conn = sqlite3.connect('../../Finances.db')
c = conn.cursor()

## Company valuation

In [5]:
prefix = 'CVALUATION__'

### Profile

In [6]:
name = prefix+'PROFILE'

In [7]:
%%time
url = "https://financialmodelingprep.com/api/v3/profile/***TICKER***?apikey=" + apikey
tab = fmp_todf(url, tickers)
a = pd.io.sql.get_schema(tab,name)
#print(a)

CPU times: user 832 ms, sys: 46.9 ms, total: 879 ms
Wall time: 13.4 s


In [8]:
#clean up
c.execute('drop table if exists '+ name)

#add primary key (can't do with pandas)
c.execute("""
CREATE TABLE "CVALUATION__PROFILE" (
"symbol" TEXT PRIMARY KEY,
"price" REAL,
"beta" REAL,
"volAvg" INTEGER,
"mktCap" INTEGER,
"lastDiv" REAL,
"range" TEXT,
"changes" REAL,
"companyName" TEXT,
"currency" TEXT,
"cik" TEXT,
"isin" TEXT,
"cusip" TEXT,
"exchange" TEXT,
"exchangeShortName" TEXT,
"industry" TEXT,
"website" TEXT,
"description" TEXT,
"ceo" TEXT,
"sector" TEXT,
"country" TEXT,
"fullTimeEmployees" TEXT,
"phone" TEXT,
"address" TEXT,
"city" TEXT,
"state" TEXT,
"zip" TEXT,
"dcfDiff" REAL,
"dcf" REAL,
"image" TEXT,
"ipoDate" TEXT,
"defaultImage" INTEGER
)""")

#put data into table
tab.to_sql(name,con=conn,index=False,if_exists='append')
c.execute('select * from '+name+ ' limit 1')
print(c.fetchall()[0][0:10],)

('CRM', 236.7, 1.25764, 9457330, 217227641000, 0.0, '115.29-284.5', -2.23, 'Salesforce.Com Inc', 'USD')


### Quote

In [9]:
name = prefix+'QUOTE'

In [10]:
%%time
url = "https://financialmodelingprep.com/api/v3/quote/***TICKER***?apikey=" + apikey
tab = fmp_todf(url, tickers)
a = pd.io.sql.get_schema(tab,name)
#print(a)

CPU times: user 669 ms, sys: 24.4 ms, total: 694 ms
Wall time: 9.49 s


In [11]:
c.execute('drop table if exists '+ name)

c.execute("""
CREATE TABLE "CVALUATION__QUOTE" (
  "symbol" TEXT PRIMARY KEY,
  "name" TEXT,
  "price" REAL,
  "changesPercentage" REAL,
  "change" REAL,
  "dayLow" REAL,
  "dayHigh" REAL,
  "yearHigh" REAL,
  "yearLow" REAL,
  "marketCap" REAL,
  "priceAvg50" REAL,
  "priceAvg200" REAL,
  "volume" INTEGER,
  "avgVolume" INTEGER,
  "exchange" TEXT,
  "open" REAL,
  "previousClose" REAL,
  "eps" REAL,
  "pe" REAL,
  "earningsAnnouncement" TEXT,
  "sharesOutstanding" INTEGER,
  "timestamp" INTEGER
)""")

#put data into table
tab.to_sql(name,con=conn,index=False,if_exists='append')
c.execute('select * from '+name+ ' limit 1')
print(c.fetchall()[0][0:10],)

('CRM', 'salesforce.com, inc.', 236.7, -0.93, -2.23, 236.2366, 240.08, 284.5, 115.29, 217227640832.0)


### Key Executives

In [12]:
name = prefix+'KEY_EXECUTIVES'

In [13]:
%%time
url = "https://financialmodelingprep.com/api/v3/key-executives/***TICKER***?apikey=" + apikey
tab = fmp_todf(url, tickers)
a = pd.io.sql.get_schema(tab,name)
#print(a)

CPU times: user 1.58 s, sys: 26.1 ms, total: 1.6 s
Wall time: 13.2 s


In [14]:
c.execute('drop table if exists '+name)
c.execute("""
CREATE TABLE "CVALUATION__KEY_EXECUTIVES" (
  "symbol" TEXT,
  "title" TEXT,
  "name" TEXT PRIMARY KEY,
  "pay" INTEGER,
  "currencyPay" TEXT,
  "gender" TEXT,
  "yearBorn" INTEGER,
  "titleSince" TEXT
)""")
tab['symbol'] = tab.index
tab.to_sql(name,con=conn,index=False,if_exists='append')
c.execute('select * from '+name+ ' limit 1')
print(c.fetchall()[0][0:10],)

('CRM', 'Co-Founder, Chairman & Chief Executive Officer', 'Mr. Marc R. Benioff', 5970000, 'USD', 'male', 1965, None)


### !!!Out for now: Search

### !!!Out for now: Financial Zip

### Financial Statements

#### Income

##### Quarterly

In [15]:
name = prefix+'FINANCIAL_STATEMENTS_INCOME_Q'

In [16]:
%%time
url = "https://financialmodelingprep.com/api/v3/income-statement/***TICKER***?period=quarter&limit=1000000000&apikey=" + apikey
tab = fmp_todf(url, tickers)
a = pd.io.sql.get_schema(tab,name)
#print(a)

CPU times: user 37.3 s, sys: 165 ms, total: 37.4 s
Wall time: 52.5 s


In [17]:
c.execute('drop table if exists '+name)
c.execute("""
CREATE TABLE "CVALUATION__FINANCIAL_STATEMENTS_INCOME_Q" (
  "date" TEXT,
  "symbol" TEXT,
  "reportedCurrency" TEXT,
  "fillingDate" TEXT,
  "acceptedDate" TEXT,
  "period" TEXT,
  "revenue" TEXT,
  "costOfRevenue" TEXT,
  "grossProfit" TEXT,
  "grossProfitRatio" REAL,
  "researchAndDevelopmentExpenses" TEXT,
  "generalAndAdministrativeExpenses" TEXT,
  "sellingAndMarketingExpenses" TEXT,
  "otherExpenses" TEXT,
  "operatingExpenses" TEXT,
  "costAndExpenses" TEXT,
  "interestExpense" REAL,
  "depreciationAndAmortization" TEXT,
  "ebitda" TEXT,
  "ebitdaratio" REAL,
  "operatingIncome" TEXT,
  "operatingIncomeRatio" REAL,
  "totalOtherIncomeExpensesNet" TEXT,
  "incomeBeforeTax" TEXT,
  "incomeBeforeTaxRatio" REAL,
  "incomeTaxExpense" TEXT,
  "netIncome" TEXT,
  "netIncomeRatio" REAL,
  "eps" REAL,
  "epsdiluted" REAL,
  "weightedAverageShsOut" TEXT,
  "weightedAverageShsOutDil" TEXT,
  "link" TEXT,
  "finalLink" TEXT,
  PRIMARY KEY (symbol,date)
)""")
tab.to_sql(name,con=conn,index=False,if_exists='append')
c.execute('select * from '+name+ ' limit 1')
print(c.fetchall()[0][0:10],)

('2020-10-31', 'CRM', 'USD', '2020-12-04', '2020-12-04 08:43:26', 'Q3', '5419000000', '1394000000', '4025000000', 0.7427569662299317)


#### Balance Sheet

##### Quarterly

In [18]:
name = prefix+'FINANCIAL_STATEMENTS_BALANCE_SHEET_Q'

In [19]:
%%time
url = "https://financialmodelingprep.com/api/v3/balance-sheet-statement/***TICKER***?period=quarter&limit=1000000000&apikey=" + apikey
tab = fmp_todf(url, tickers)
a = pd.io.sql.get_schema(tab,name)
#print(a)

CPU times: user 52.1 s, sys: 237 ms, total: 52.3 s
Wall time: 1min 7s


In [20]:
c.execute('drop table if exists '+name)
c.execute("""
CREATE TABLE "CVALUATION__FINANCIAL_STATEMENTS_BALANCE_SHEET_Q" (
  "date" TEXT,
  "symbol" TEXT,
  "reportedCurrency" TEXT,
  "fillingDate" TEXT,
  "acceptedDate" TEXT,
  "period" TEXT,
  "cashAndCashEquivalents" TEXT,
  "shortTermInvestments" TEXT,
  "cashAndShortTermInvestments" TEXT,
  "netReceivables" TEXT,
  "inventory" REAL,
  "otherCurrentAssets" REAL,
  "totalCurrentAssets" TEXT,
  "propertyPlantEquipmentNet" TEXT,
  "goodwill" TEXT,
  "intangibleAssets" TEXT,
  "goodwillAndIntangibleAssets" TEXT,
  "longTermInvestments" REAL,
  "taxAssets" TEXT,
  "otherNonCurrentAssets" REAL,
  "totalNonCurrentAssets" TEXT,
  "otherAssets" REAL,
  "totalAssets" TEXT,
  "accountPayables" TEXT,
  "shortTermDebt" TEXT,
  "taxPayables" REAL,
  "deferredRevenue" TEXT,
  "otherCurrentLiabilities" TEXT,
  "totalCurrentLiabilities" TEXT,
  "longTermDebt" TEXT,
  "deferredRevenueNonCurrent" REAL,
  "deferredTaxLiabilitiesNonCurrent" REAL,
  "otherNonCurrentLiabilities" TEXT,
  "totalNonCurrentLiabilities" TEXT,
  "otherLiabilities" TEXT,
  "totalLiabilities" TEXT,
  "commonStock" REAL,
  "retainedEarnings" TEXT,
  "accumulatedOtherComprehensiveIncomeLoss" REAL,
  "othertotalStockholdersEquity" REAL,
  "totalStockholdersEquity" TEXT,
  "totalLiabilitiesAndStockholdersEquity" TEXT,
  "totalInvestments" REAL,
  "totalDebt" TEXT,
  "netDebt" TEXT,
  "link" TEXT,
  "finalLink" TEXT,
  PRIMARY KEY (symbol,date)
)""")
tab.to_sql(name,con=conn,index=False,if_exists='append')
c.execute('select * from '+name+ ' limit 1')
print(c.fetchall()[0][0:10],)

('2020-10-31', 'CRM', 'USD', '2020-12-04', '2020-12-04 08:43:26', 'Q3', '3724000000', '5768000000', '9492000000', '3316000000')


#### Cash Flow Statement

##### Quarterly

In [21]:
name = prefix+'FINANCIAL_STATEMENTS_CASH_FLOW_Q'

In [22]:
%%time
url = "https://financialmodelingprep.com/api/v3/cash-flow-statement/***TICKER***?period=quarter&limit=1000000000&apikey=" + apikey
tab = fmp_todf(url, tickers)
a = pd.io.sql.get_schema(tab,name)
#print(a)

CPU times: user 38 s, sys: 162 ms, total: 38.2 s
Wall time: 52.5 s


In [23]:
c.execute('drop table if exists '+name)
c.execute("""
CREATE TABLE "CVALUATION__FINANCIAL_STATEMENTS_CASH_FLOW_Q" (
  "date" TEXT,
  "symbol" TEXT,
  "reportedCurrency" TEXT,
  "fillingDate" TEXT,
  "acceptedDate" TEXT,
  "period" TEXT,
  "netIncome" TEXT,
  "depreciationAndAmortization" TEXT,
  "deferredIncomeTax" REAL,
  "stockBasedCompensation" TEXT,
  "changeInWorkingCapital" TEXT,
  "accountsReceivables" TEXT,
  "inventory" REAL,
  "accountsPayables" TEXT,
  "otherWorkingCapital" TEXT,
  "otherNonCashItems" TEXT,
  "netCashProvidedByOperatingActivities" TEXT,
  "investmentsInPropertyPlantAndEquipment" REAL,
  "acquisitionsNet" REAL,
  "purchasesOfInvestments" TEXT,
  "salesMaturitiesOfInvestments" TEXT,
  "otherInvestingActivites" REAL,
  "netCashUsedForInvestingActivites" REAL,
  "debtRepayment" TEXT,
  "commonStockIssued" REAL,
  "commonStockRepurchased" REAL,
  "dividendsPaid" REAL,
  "otherFinancingActivites" REAL,
  "netCashUsedProvidedByFinancingActivities" REAL,
  "effectOfForexChangesOnCash" REAL,
  "netChangeInCash" TEXT,
  "cashAtEndOfPeriod" TEXT,
  "cashAtBeginningOfPeriod" TEXT,
  "operatingCashFlow" TEXT,
  "capitalExpenditure" TEXT,
  "freeCashFlow" TEXT,
  "link" TEXT,
  "finalLink" TEXT,
  PRIMARY KEY (symbol,date)
)""")
tab.to_sql(name,con=conn,index=False,if_exists='append')
c.execute('select * from '+name+ ' limit 1')
print(c.fetchall()[0][0:10],)

('2020-10-31', 'CRM', 'USD', '2020-12-04', '2020-12-04 08:43:26', 'Q3', '1081000000', '670000000', 0.0, '566000000')


### !!!Out for now: Financial Statements List

### Growth statements (Annual)

#### Income 

In [24]:
name = prefix+'GROWTH_STATEMENTS_INCOME_Y'

In [25]:
%%time
url = "https://financialmodelingprep.com/api/v3/income-statement-growth/***TICKER***?limit=1000000000&apikey=" + apikey
tab = fmp_todf(url, tickers)
a = pd.io.sql.get_schema(tab,name)
#print(a)

CPU times: user 4.13 s, sys: 36.1 ms, total: 4.17 s
Wall time: 15 s


In [26]:
c.execute('drop table if exists '+name)
c.execute("""
CREATE TABLE "CVALUATION__GROWTH_STATEMENTS_INCOME_Y" (
  "date" TEXT,
  "symbol" TEXT,
  "period" TEXT,
  "growthRevenue" REAL,
  "growthCostOfRevenue" REAL,
  "growthGrossProfit" REAL,
  "growthGrossProfitRatio" REAL,
  "growthResearchAndDevelopmentExpenses" REAL,
  "growthGeneralAndAdministrativeExpenses" REAL,
  "growthSellingAndMarketingExpenses" REAL,
  "growthOtherExpenses" REAL,
  "growthOperatingExpenses" REAL,
  "growthCostAndExpenses" REAL,
  "growthInterestExpense" REAL,
  "growthDepreciationAndAmortization" REAL,
  "growthEBITDA" REAL,
  "growthEBITDARatio" REAL,
  "growthOperatingIncome" REAL,
  "growthOperatingIncomeRatio" REAL,
  "growthTotalOtherIncomeExpensesNet" REAL,
  "growthIncomeBeforeTax" REAL,
  "growthIncomeBeforeTaxRatio" REAL,
  "growthIncomeTaxExpense" REAL,
  "growthNetIncome" REAL,
  "growthNetIncomeRatio" REAL,
  "growthEPS" REAL,
  "growthEPSDiluted" REAL,
  "growthWeightedAverageShsOut" REAL,
  "growthWeightedAverageShsOutDil" REAL,
  PRIMARY KEY (symbol,date)
)""")

tab.to_sql(name,con=conn,index=False,if_exists='append')
c.execute('select * from '+name+ ' limit 1')
print(c.fetchall()[0][0:10],)

('2020-01-31', 'CRM', 'FY', 0.2873061285950911, 0.22718052738336714, 0.3084121655986166, 0.01639477150673828, 0.46659597030752914, 0.2980002796811635, 0.30771767810026385)


#### Balance Sheet

In [27]:
name = prefix+'GROWTH_STATEMENTS_BALANCE_SHEET_Y'

In [28]:
%%time
url = "https://financialmodelingprep.com/api/v3/balance-sheet-statement-growth/***TICKER***?limit=1000000000&apikey=" + apikey
tab = fmp_todf(url, tickers)
a = pd.io.sql.get_schema(tab,name)
#print(a)

CPU times: user 5.3 s, sys: 49.4 ms, total: 5.35 s
Wall time: 17.4 s


In [29]:
c.execute('drop table if exists '+name)
c.execute("""
CREATE TABLE "CVALUATION__GROWTH_STATEMENTS_BALANCE_SHEET_Y" (
"date" TEXT,
  "symbol" TEXT,
  "period" TEXT,
  "growthCashAndCashEquivalents" REAL,
  "growthShortTermInvestments" REAL,
  "growthCashAndShortTermInvestments" REAL,
  "growthNetReceivables" REAL,
  "growthInventory" REAL,
  "growthOtherCurrentAssets" REAL,
  "growthTotalCurrentAssets" REAL,
  "growthPropertyPlantEquipmentNet" REAL,
  "growthGoodwill" REAL,
  "growthIntangibleAssets" REAL,
  "growthGoodwillAndIntangibleAssets" REAL,
  "growthLongTermInvestments" REAL,
  "growthTaxAssets" REAL,
  "growthOtherNonCurrentAssets" REAL,
  "growthTotalNonCurrentAssets" REAL,
  "growthOtherAssets" REAL,
  "growthTotalAssets" REAL,
  "growthAccountPayables" REAL,
  "growthShortTermDebt" REAL,
  "growthTaxPayables" REAL,
  "growthDeferredRevenue" REAL,
  "growthOtherCurrentLiabilities" REAL,
  "growthTotalCurrentLiabilities" REAL,
  "growthLongTermDebt" REAL,
  "growthDeferredRevenueNonCurrent" REAL,
  "growthDeferrredTaxLiabilitiesNonCurrent" REAL,
  "growthOtherNonCurrentLiabilities" REAL,
  "growthTotalNonCurrentLiabilities" REAL,
  "growthOtherLiabilities" REAL,
  "growthTotalLiabilities" REAL,
  "growthCommonStock" REAL,
  "growthRetainedEarnings" REAL,
  "growthAccumulatedOtherComprehensiveIncomeLoss" REAL,
  "growthOthertotalStockholdersEquity" REAL,
  "growthTotalStockholdersEquity" REAL,
  "growthTotalLiabilitiesAndStockholdersEquity" REAL,
  "growthTotalInvestments" REAL,
  "growthTotalDebt" REAL,
  "growthNetDebt" REAL,
  PRIMARY KEY (symbol,date)
)""")

tab.to_sql(name,con=conn,index=False,if_exists='append')
c.execute('select * from '+name+ ' limit 1')
print(c.fetchall()[0][0:10],)

('2020-01-31', 'CRM', 'FY', 0.5530161109029599, 1.2725642558278543, 0.8302625518194381, 0.24299719887955182, 0.0, 0.4562798092209857, 0.4942431901151362)


#### Cash Flow

In [30]:
name = prefix+'GROWTH_STATEMENTS_CASH_FLOW_Y'

In [31]:
%%time
url = "https://financialmodelingprep.com/api/v3/cash-flow-statement-growth/***TICKER***?limit=1000000000&apikey=" + apikey
tab = fmp_todf(url, tickers)
a = pd.io.sql.get_schema(tab,name)
print(a)

CREATE TABLE "CVALUATION__GROWTH_STATEMENTS_CASH_FLOW_Y" (
"date" TEXT,
  "symbol" TEXT,
  "period" TEXT,
  "growthNetIncome" REAL,
  "growthDepreciationAndAmortization" REAL,
  "growthDeferredIncomeTax" REAL,
  "growthStockBasedCompensation" REAL,
  "growthChangeInWorkingCapital" REAL,
  "growthAccountsReceivables" REAL,
  "growthInventory" REAL,
  "growthAccountsPayables" REAL,
  "growthOtherWorkingCapital" REAL,
  "growthOtherNonCashItems" REAL,
  "growthNetCashProvidedByOperatingActivites" REAL,
  "growthInvestmentsInPropertyPlantAndEquipment" REAL,
  "growthAcquisitionsNet" REAL,
  "growthPurchasesOfInvestments" REAL,
  "growthSalesMaturitiesOfInvestments" REAL,
  "growthOtherInvestingActivites" REAL,
  "growthNetCashUsedForInvestingActivites" REAL,
  "growthDebtRepayment" REAL,
  "growthCommonStockIssued" REAL,
  "growthCommonStockRepurchased" REAL,
  "growthDividendsPaid" REAL,
  "growthOtherFinancingActivites" REAL,
  "growthNetCashUsedProvidedByFinancingActivities" REAL,
  "gr

In [32]:
c.execute('drop table if exists '+name)
c.execute("""
CREATE TABLE "CVALUATION__GROWTH_STATEMENTS_CASH_FLOW_Y" (
"date" TEXT,
  "symbol" TEXT,
  "period" TEXT,
  "growthNetIncome" REAL,
  "growthDepreciationAndAmortization" REAL,
  "growthDeferredIncomeTax" REAL,
  "growthStockBasedCompensation" REAL,
  "growthChangeInWorkingCapital" REAL,
  "growthAccountsReceivables" REAL,
  "growthInventory" REAL,
  "growthAccountsPayables" REAL,
  "growthOtherWorkingCapital" REAL,
  "growthOtherNonCashItems" REAL,
  "growthNetCashProvidedByOperatingActivites" REAL,
  "growthInvestmentsInPropertyPlantAndEquipment" REAL,
  "growthAcquisitionsNet" REAL,
  "growthPurchasesOfInvestments" REAL,
  "growthSalesMaturitiesOfInvestments" REAL,
  "growthOtherInvestingActivites" REAL,
  "growthNetCashUsedForInvestingActivites" REAL,
  "growthDebtRepayment" REAL,
  "growthCommonStockIssued" REAL,
  "growthCommonStockRepurchased" REAL,
  "growthDividendsPaid" REAL,
  "growthOtherFinancingActivites" REAL,
  "growthNetCashUsedProvidedByFinancingActivities" REAL,
  "growthEffectOfForexChangesOnCash" REAL,
  "growthNetChangeInCash" REAL,
  "growthCashAtEndOfPeriod" REAL,
  "growthCashAtBeginningOfPeriod" REAL,
  "growthOperatingCashFlow" REAL,
  "growthCapitalExpenditure" REAL,
  "growthFreeCashFlow" REAL,
  PRIMARY KEY (symbol,date)
)""")

tab.to_sql(name,con=conn,index=False,if_exists='append')
c.execute('select * from '+name+ ' limit 1')
print(c.fetchall()[0][0:10],)

('2020-01-31', 'CRM', 'FY', -0.8864864864864865, 1.2193347193347193, 0.0, -0.8409976617303195, -19.98876404494382, 0.08342361863488625, 0.0)


### !!!Out for now: International Filings

### !!!Out for now: Financial Statements As Reported

### Financial Ratios

##### Quarterly

In [33]:
name = prefix+'FINANCIAL_RATIOS_Q'

In [34]:
%%time
url = "https://financialmodelingprep.com/api/v3/ratios/***TICKER***?period=quarter&limit=1000000000&apikey=" + apikey
tab = fmp_todf(url, tickers)
a = pd.io.sql.get_schema(tab,name)
#print(a)

CPU times: user 51.3 s, sys: 223 ms, total: 51.6 s
Wall time: 1min 14s


In [35]:
c.execute('drop table if exists '+name)
c.execute("""
CREATE TABLE "CVALUATION__FINANCIAL_RATIOS_Q" (
"symbol" TEXT,
  "date" TEXT,
  "currentRatio" REAL,
  "quickRatio" REAL,
  "cashRatio" REAL,
  "daysOfSalesOutstanding" REAL,
  "daysOfInventoryOutstanding" REAL,
  "operatingCycle" REAL,
  "daysOfPayablesOutstanding" REAL,
  "cashConversionCycle" REAL,
  "grossProfitMargin" REAL,
  "operatingProfitMargin" REAL,
  "pretaxProfitMargin" REAL,
  "netProfitMargin" REAL,
  "effectiveTaxRate" REAL,
  "returnOnAssets" REAL,
  "returnOnEquity" REAL,
  "returnOnCapitalEmployed" REAL,
  "netIncomePerEBT" REAL,
  "ebtPerEbit" REAL,
  "ebitPerRevenue" REAL,
  "debtRatio" REAL,
  "debtEquityRatio" REAL,
  "longTermDebtToCapitalization" REAL,
  "totalDebtToCapitalization" REAL,
  "interestCoverage" REAL,
  "cashFlowToDebtRatio" REAL,
  "companyEquityMultiplier" REAL,
  "receivablesTurnover" REAL,
  "payablesTurnover" REAL,
  "inventoryTurnover" REAL,
  "fixedAssetTurnover" REAL,
  "assetTurnover" REAL,
  "operatingCashFlowPerShare" REAL,
  "freeCashFlowPerShare" REAL,
  "cashPerShare" REAL,
  "payoutRatio" REAL,
  "operatingCashFlowSalesRatio" REAL,
  "freeCashFlowOperatingCashFlowRatio" REAL,
  "cashFlowCoverageRatios" REAL,
  "shortTermCoverageRatios" REAL,
  "capitalExpenditureCoverageRatio" REAL,
  "dividendPaidAndCapexCoverageRatio" REAL,
  "dividendPayoutRatio" REAL,
  "priceBookValueRatio" REAL,
  "priceToBookRatio" REAL,
  "priceToSalesRatio" REAL,
  "priceEarningsRatio" REAL,
  "priceToFreeCashFlowsRatio" REAL,
  "priceToOperatingCashFlowsRatio" REAL,
  "priceCashFlowRatio" REAL,
  "priceEarningsToGrowthRatio" REAL,
  "priceSalesRatio" REAL,
  "dividendYield" REAL,
  "enterpriseValueMultiple" REAL,
  "priceFairValue" REAL,
  PRIMARY KEY (symbol,date)
)""")

tab.to_sql(name,con=conn,index=False,if_exists='append')
c.execute('select * from '+name+ ' limit 1')
print(c.fetchall()[0][0:10],)

('CRM', '2020-10-31', 1.2216772927905837, 1.0469184240640836, 0.30439758051332355, 223.35117180291567, 0.0, 55.07289167743126, 226.29124820659973, -171.2183565291685)


### Enterprise Value

##### Quarterly

In [36]:
name = prefix+'ENTERPRISE_VALUE_Q'

In [37]:
%%time
url = "https://financialmodelingprep.com/api/v3/enterprise-values/***TICKER***?period=quarter&limit=1000000000&apikey=" + apikey
tab = fmp_todf(url, tickers)
a = pd.io.sql.get_schema(tab,name)
#print(a)

CPU times: user 14.8 s, sys: 67 ms, total: 14.8 s
Wall time: 29.8 s


In [38]:
c.execute('drop table if exists '+name)
c.execute("""
CREATE TABLE "CVALUATION__ENTERPRISE_VALUE_Q" (
  "symbol" TEXT,
  "date" TEXT,
  "stockPrice" REAL,
  "numberOfShares" TEXT,
  "marketCapitalization" TEXT,
  "minusCashAndCashEquivalents" TEXT,
  "addTotalDebt" TEXT,
  "enterpriseValue" TEXT,
  PRIMARY KEY (symbol,date)
  
)""")

tab.to_sql(name,con=conn,index=False,if_exists='append')
c.execute('select * from '+name+ ' limit 1')
print(c.fetchall()[0][0:10],)

('CRM', '2020-10-31', 232.270004, '911000000', '211597973644', '3724000000', '8440000000', '216313973644')


### Key Metrics

##### Quarterly

In [39]:
name = prefix+'KEY_METRICS_Q'

In [40]:
%%time
url = "https://financialmodelingprep.com/api/v3/key-metrics/***TICKER***?period=quarter&limit=1000000000&apikey=" + apikey
tab = fmp_todf(url, tickers)
a = pd.io.sql.get_schema(tab,name)
#print(a)

CPU times: user 57.2 s, sys: 287 ms, total: 57.5 s
Wall time: 1min 21s


In [41]:
c.execute('drop table if exists '+name)
c.execute("""
CREATE TABLE "CVALUATION__KEY_METRICS_Q" (
"symbol" TEXT,
  "date" TEXT,
  "revenuePerShare" REAL,
  "netIncomePerShare" REAL,
  "operatingCashFlowPerShare" REAL,
  "freeCashFlowPerShare" REAL,
  "cashPerShare" REAL,
  "bookValuePerShare" REAL,
  "tangibleBookValuePerShare" REAL,
  "shareholdersEquityPerShare" REAL,
  "interestDebtPerShare" TEXT,
  "marketCap" TEXT,
  "enterpriseValue" TEXT,
  "peRatio" TEXT,
  "priceToSalesRatio" REAL,
  "pocfratio" REAL,
  "pfcfRatio" REAL,
  "pbRatio" REAL,
  "ptbRatio" REAL,
  "evToSales" REAL,
  "enterpriseValueOverEBITDA" REAL,
  "evToOperatingCashFlow" REAL,
  "evToFreeCashFlow" REAL,
  "earningsYield" REAL,
  "freeCashFlowYield" REAL,
  "debtToEquity" REAL,
  "debtToAssets" REAL,
  "netDebtToEBITDA" TEXT,
  "currentRatio" TEXT,
  "interestCoverage" TEXT,
  "incomeQuality" TEXT,
  "dividendYield" REAL,
  "payoutRatio" TEXT,
  "salesGeneralAndAdministrativeToRevenue" REAL,
  "researchAndDdevelopementToRevenue" REAL,
  "intangiblesToTotalAssets" REAL,
  "capexToOperatingCashFlow" TEXT,
  "capexToRevenue" TEXT,
  "capexToDepreciation" TEXT,
  "stockBasedCompensationToRevenue" REAL,
  "grahamNumber" REAL,
  "roic" REAL,
  "returnOnTangibleAssets" REAL,
  "grahamNetNet" REAL,
  "workingCapital" TEXT,
  "tangibleAssetValue" TEXT,
  "netCurrentAssetValue" TEXT,
  "investedCapital" TEXT,
  "averageReceivables" INTEGER,
  "averagePayables" INTEGER,
  "averageInventory" INTEGER,
  "daysSalesOutstanding" REAL,
  "daysPayablesOutstanding" TEXT,
  "daysOfInventoryOnHand" TEXT,
  "receivablesTurnover" REAL,
  "payablesTurnover" TEXT,
  "inventoryTurnover" REAL,
  "roe" REAL,
  "capexPerShare" TEXT,
  PRIMARY KEY (symbol,date)
)""")

tab.to_sql(name,con=conn,index=False,if_exists='append')
c.execute('select * from '+name+ ' limit 1')
print(c.fetchall()[0][0:10],)

('CRM', '2020-10-31', 5.94840834248079, 1.1866081229418222, 0.3721185510428101, 0.23600439077936333, 4.08781558726674, 44.24807903402854, 31.201975850713502, 44.24807903402854)


### Financial Growth 

##### Quarterly

In [42]:
name = prefix+'FINANCIAL_GROWTH_Q'

In [43]:
%%time
url = "https://financialmodelingprep.com/api/v3/key-metrics/***TICKER***?period=quarter&limit=1000000000&apikey=" + apikey
tab = fmp_todf(url, tickers)
a = pd.io.sql.get_schema(tab,name)
#print(a)

CPU times: user 57.5 s, sys: 307 ms, total: 57.8 s
Wall time: 1min 19s


In [44]:
c.execute('drop table if exists '+name)
c.execute("""
CREATE TABLE "CVALUATION__FINANCIAL_GROWTH_Q" (
 "symbol" TEXT,
  "date" TEXT,
  "revenuePerShare" REAL,
  "netIncomePerShare" REAL,
  "operatingCashFlowPerShare" REAL,
  "freeCashFlowPerShare" REAL,
  "cashPerShare" REAL,
  "bookValuePerShare" REAL,
  "tangibleBookValuePerShare" REAL,
  "shareholdersEquityPerShare" REAL,
  "interestDebtPerShare" TEXT,
  "marketCap" TEXT,
  "enterpriseValue" TEXT,
  "peRatio" REAL,
  "priceToSalesRatio" REAL,
  "pocfratio" REAL,
  "pfcfRatio" REAL,
  "pbRatio" REAL,
  "ptbRatio" REAL,
  "evToSales" REAL,
  "enterpriseValueOverEBITDA" REAL,
  "evToOperatingCashFlow" REAL,
  "evToFreeCashFlow" REAL,
  "earningsYield" REAL,
  "freeCashFlowYield" REAL,
  "debtToEquity" REAL,
  "debtToAssets" REAL,
  "netDebtToEBITDA" TEXT,
  "currentRatio" REAL,
  "interestCoverage" TEXT,
  "incomeQuality" TEXT,
  "dividendYield" REAL,
  "payoutRatio" TEXT,
  "salesGeneralAndAdministrativeToRevenue" REAL,
  "researchAndDdevelopementToRevenue" REAL,
  "intangiblesToTotalAssets" REAL,
  "capexToOperatingCashFlow" TEXT,
  "capexToRevenue" TEXT,
  "capexToDepreciation" TEXT,
  "stockBasedCompensationToRevenue" REAL,
  "grahamNumber" REAL,
  "roic" REAL,
  "returnOnTangibleAssets" REAL,
  "grahamNetNet" REAL,
  "workingCapital" TEXT,
  "tangibleAssetValue" TEXT,
  "netCurrentAssetValue" TEXT,
  "investedCapital" TEXT,
  "averageReceivables" INTEGER,
  "averagePayables" INTEGER,
  "averageInventory" INTEGER,
  "daysSalesOutstanding" REAL,
  "daysPayablesOutstanding" REAL,
  "daysOfInventoryOnHand" REAL,
  "receivablesTurnover" TEXT,
  "payablesTurnover" TEXT,
  "inventoryTurnover" REAL,
  "roe" REAL,
  "capexPerShare" REAL,
  PRIMARY KEY (symbol,date)
)""")

tab.to_sql(name,con=conn,index=False,if_exists='append')
c.execute('select * from '+name+ ' limit 1')
print(c.fetchall()[0][0:10],)

('CRM', '2020-10-31', 5.94840834248079, 1.1866081229418222, 0.3721185510428101, 0.23600439077936333, 4.08781558726674, 44.24807903402854, 31.201975850713502, 44.24807903402854)


### Rating

In [59]:
son = get_json('https://financialmodelingprep.com/api/v3/stock/list?apikey='+apikey)
tickers2 = pd.DataFrame(son)
tickers2['exchange'].value_counts()

LSE                        4502
HKSE                       3151
New York Stock Exchange    2623
Nasdaq Global Select       2270
ASX                        1947
Toronto                    1759
NSE                        1632
Nasdaq                     1550
NYSE Arca                  1540
NASDAQ Capital Market      1333
Other OTC                  1008
XETRA                       909
Paris                       884
NASDAQ Global Market        847
NYSE                        704
SIX                         528
NYSE American               360
NYSEArca                    346
BATS Exchange               258
YHD                         256
MCX                         216
OSE                         192
NasdaqGM                    169
Brussels                    167
BATS                        145
Amsterdam                   137
NMS                          75
Lisbon                       52
Irish                        39
NCM                          32
                             21
Oslo    